<a href="https://colab.research.google.com/github/dcpatton/Image-Classification/blob/master/cifar10_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective



> Just a simple example of image classification with a CNN in TensorFlow



In [26]:
import tensorflow as tf
import random

seed = 52
tf.random.set_seed(seed)
random.seed(seed)

tf.__version__

'2.3.0'

# Get the data

In [27]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [28]:
print(train_images.shape)
print(test_images.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


# Model

In [29]:
normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
normalizer.adapt(train_images)

In [30]:
tf.keras.backend.clear_session()
from tensorflow.keras.layers import Conv2D, Flatten, MaxPool2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomRotation, RandomCrop
from tensorflow.keras.layers.experimental.preprocessing import RandomContrast, RandomFlip, RandomZoom, RandomTranslation
from tensorflow.keras import Input, Model

inp = Input(shape=(32, 32, 3), name='inp', dtype=tf.float32)
# x = RandomContrast(0.05, seed=seed)(inp)
x = RandomRotation(0.05, seed=seed)(inp)
x = RandomFlip(mode='horizontal', seed=seed)(x)
x = RandomZoom(height_factor=0.15, width_factor=0.15, seed=seed)(x)
x = RandomTranslation(height_factor=0.1, width_factor=0.1, seed=seed)(x)
x = RandomCrop(30, 30, seed=seed)(x)
# x = Rescaling(scale=1./255, offset=0.)(x)
x = normalizer(x)

x = Conv2D(32, 3, activation='elu', padding='valid')(x)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation='elu', padding='valid')(x)
x = BatchNormalization()(x)
x = MaxPool2D()(x)
x = Conv2D(128, 3, activation='elu')(x)
x = BatchNormalization()(x)
x = Conv2D(256, 3, activation='elu')(x)
x = BatchNormalization()(x)
x = MaxPool2D()(x)
x = Conv2D(512, 3, activation='elu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(1024, 3, activation='elu')(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.25)(x)
out = Dense(10, activation='softmax', name='out')(x)

model = Model(inputs=[inp], outputs=[out])

model.compile(optimizer=tf.keras.optimizers.Adam(amsgrad=True),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), 
              metrics=['acc'])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             [(None, 32, 32, 3)]       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 32, 32, 3)         0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 32, 32, 3)         0         
_________________________________________________________________
random_zoom (RandomZoom)     (None, 32, 32, 3)         0         
_________________________________________________________________
random_translation (RandomTr (None, 32, 32, 3)         0         
_________________________________________________________________
random_crop (RandomCrop)     (None, 30, 30, 3)         0         
_________________________________________________________________
normalization (Normalization (None, 30, 30, 3)        

# Training

In [31]:
filepath = 'model.h5'
mc = tf.keras.callbacks.ModelCheckpoint(filepath, save_best_only=True, 
                                        save_weights_only=True)
es = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1)

history = model.fit(train_images, train_labels, epochs=200, 
                    validation_data=(test_images, test_labels), 
                    callbacks=[mc, es], verbose=1, batch_size=32)

Epoch 1/200
1563/1563 [==============================] - 16s 10ms/step - loss: 1.7482 - acc: 0.3876 - val_loss: 1.3442 - val_acc: 0.5149
Epoch 2/200
1563/1563 [==============================] - 15s 10ms/step - loss: 1.3253 - acc: 0.5276 - val_loss: 1.0702 - val_acc: 0.6223
Epoch 3/200
1563/1563 [==============================] - 16s 10ms/step - loss: 1.1234 - acc: 0.6041 - val_loss: 1.0185 - val_acc: 0.6473
Epoch 4/200
1563/1563 [==============================] - 16s 10ms/step - loss: 0.9961 - acc: 0.6550 - val_loss: 0.8613 - val_acc: 0.7000
Epoch 5/200
1563/1563 [==============================] - 15s 10ms/step - loss: 0.9029 - acc: 0.6851 - val_loss: 0.7941 - val_acc: 0.7303
Epoch 6/200
1563/1563 [==============================] - 15s 10ms/step - loss: 0.8433 - acc: 0.7112 - val_loss: 0.7325 - val_acc: 0.7585
Epoch 7/200
1563/1563 [==============================] - 16s 10ms/step - loss: 0.7853 - acc: 0.7292 - val_loss: 0.7022 - val_acc: 0.7708
Epoch 8/200
1563/1563 [==================

# Evaluation

In [32]:
model.load_weights(filepath)
metrics = model.evaluate(test_images, test_labels, verbose=0)
print('Loss = ' + str(metrics[0]))
print('Accuracy = ' + str(metrics[1]))

Loss = 0.4390917122364044
Accuracy = 0.859499990940094


Per [benchmark.ai](https://benchmarks.ai/cifar-10), the SOTA is 99.37%.

Loss = 0.4390917122364044

Accuracy = 0.859499990940094